In [1]:
import pandas as pd
import re
import requests

In [7]:
separator = r'\', \''
df = pd.read_csv('Mosru_dump_all_pages.csv',encoding='utf-8', sep=separator)

C:\Programms\Anaconda\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [19]:
def clean_string(string):
    return re.sub('\'','',string)

In [8]:
df.head()

,'Сообщение о проблеме «Несоблюдение требований к размещению информационных вывесок» по адресу Снежная улица д.26,Выше второго этажа,Ларин А.С.,начальник Объединения административно-технических инспекций города Москвы,16 Ноября 2020 в 16:31'
0,'Сообщение о проблеме «Неубранная городская те...,По адресу Снежная д24 расположена музыкальная ...,Кучма А.А.,глава управы района Свиблово города Москвы,05 Мая 2020 в 15:45'
1,'Сообщение о проблеме «Неубранная городская те...,После проведения работ на кабельной канализаци...,Ларин А.С.,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57'
2,'Сообщение о проблеме «Неубранная городская те...,Очистите опору освещения. Приведите в надлежащ...,Кучма А.А.,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54'
3,'Сообщение о проблеме «Захламление территории»...,Более двух недель лежит куча грунта в перемешк...,Сержантов С.А.,и.о. главы управы района Свиблово города Москвы,24 Августа 2019 в 15:39'
4,'Сообщение о проблеме «Захламление территории»...,"Осколки бордюрного камня спрятаны за дерево, в...",Киселев В.В.,глава управы района Свиблово города Москвы,05 Мая 2019 в 09:11'


In [20]:
df['16 Ноября 2020 в 16:31'] = df['16 Ноября 2020 в 16:31\''].apply(clean_string)

In [22]:
dictOne = {"Тема обращения":[],
           "Суть обращения":[],
           "Чиновник":[],
           "Должность":[],
           "Дата подачи заявления":[]}
print(pd.DataFrame(dictOne))

Empty DataFrame
Columns: [Column A, Column B, Column C]
Index: []


In [21]:
df['16 Ноября 2020 в 16:31'][:5]

0        05 Мая 2020 в 15:45
1      01 Марта 2020 в 16:57
2     16 Января 2020 в 13:54
3    24 Августа 2019 в 15:39
4        05 Мая 2019 в 09:11
Name: 16 Ноября 2020 в 16:31, dtype: object